In [30]:
# diary_project.py

class DiaryNode:
    def __init__(self, id, date, title, entry, next_id):
        self.id = id
        self.date = date
        self.title = title
        self.entry = entry
        self.next_id = next_id
        self.next = None  # For linked list pointer

    def __str__(self):
        return f"{self.date} - {self.title}\n{self.entry}\n"


class DiaryLinkedList:
    def __init__(self):
        self.head = None

    def insert_node(self, node):
        # This method will be handled after linking all entries
        if not self.head:                           # To insert a new node after all the existing nodes
          self.head = node
          return
        current = self.head
        while current.next:
          current = current.next
        current.next = node
        current.next_id = node.id


    def print_diary(self):
        current = self.head
        while current:
            print(current)
            current = current.next

    def search(self, keyword):
        results = []
        current = self.head
        while current:
            if keyword.lower() in current.title.lower() or keyword.lower() in current.entry.lower():
                results.append(current)
            current = current.next
        return results

    def insert_after_date(self, date, new_node):
        # Search by date and insert new_node after it   #To insert a new node after the node with the date
        current = self.head
        while current:
            if current.date == date:
                new_node.next = current.next
                current.next = new_node
                new_node.next_id = current.next_id
                current.next_id = new_node.id
                return True
            current = current.next
        return False     # There is no date that you are searching!


    def save_to_file(self, filename):
        with open(filename, "w") as f:
            current = self.head
            while current:
                f.write(f"{current}\n")
                current = current.next


def parse_diary_file(filepath):
    diary_pages = {}
    with open(filepath, "r") as f:
        content = f.read().strip().split('---')
        for entry_text in content:
            lines = entry_text.strip().split("\n")
            if len(lines) < 5:
                continue
            id = int(lines[0].split(": ")[1])
            date = lines[1].split(": ")[1]
            title = lines[2].split(": ")[1]
            entry = lines[3].split(": ")[1]
            next_id_str = lines[4].split(": ")[1]
            next_id = int(next_id_str) if next_id_str != "NULL" else None
            node = DiaryNode(id, date, title, entry, next_id)
            diary_pages[id] = node
    return diary_pages


def build_linked_list(diary_pages):
    # Build linked list from parsed dictionary
    id_to_node = diary_pages
    visited = set()

    # Find starting node (entry that is not pointed to by anyone)
    pointed_to = set(node.next_id for node in id_to_node.values() if node.next_id is not None)
    starting_nodes = [node for node_id, node in id_to_node.items() if node_id not in pointed_to]

    if not starting_nodes:
        print("No unique starting node found.")
        return None

    head = starting_nodes[0]
    current = head
    visited.add(current.id)

    while current and current.next_id is not None:
        next_node = id_to_node.get(current.next_id)
        if not next_node or next_node.id in visited:
            break
        current.next = next_node
        visited.add(next_node.id)
        current = next_node

    diary = DiaryLinkedList()
    diary.head = head
    return diary


if __name__ == "__main__":
    diary_pages = parse_diary_file("/content/diary_pages.txt")
    diary = build_linked_list(diary_pages)
    if diary:
        diary.print_diary()

        keyword = input("\nSearch keyword: ")
        results = diary.search(keyword)
        print(f"\nSearch results for '{keyword}':")
        for result in results:
            print(result)


2022-05-05 - Rainy Day
Stayed home and listened to the rain. Peaceful.

2022-05-07 - Weekend Hike
Hiked the Twin Hills trail today. Beautiful views!

2022-05-10 - Coffee Thoughts
Met an old friend over coffee. Unexpected joy.


Search keyword: h

Search results for 'h':
2022-05-05 - Rainy Day
Stayed home and listened to the rain. Peaceful.

2022-05-07 - Weekend Hike
Hiked the Twin Hills trail today. Beautiful views!

2022-05-10 - Coffee Thoughts
Met an old friend over coffee. Unexpected joy.



In [31]:
#Example for insert_node function, inserting a new node after all the existing nodes
diary.insert_node(DiaryNode(105, "2024-01-01", "Rainy Night", "Met an old friend over coffee. Unexpected joy.", None))
diary.print_diary()


2022-05-05 - Rainy Day
Stayed home and listened to the rain. Peaceful.

2022-05-07 - Weekend Hike
Hiked the Twin Hills trail today. Beautiful views!

2022-05-10 - Coffee Thoughts
Met an old friend over coffee. Unexpected joy.

2024-01-01 - Rainy Night
Met an old friend over coffee. Unexpected joy.



In [32]:

#Example for insert_after_date function, inserting a new node after searching and having found the date! If not, the new node will not be inserted!
#Inserting the new node after having found the date
diary.insert_after_date("2022-05-03", DiaryNode(104, "2022-05-04", "Family Reunion", "My family came to Chiang Mai. Unexpected joy.", None))

#Inserting the new node however the date could not be found! Inserting incomplete!
diary.insert_after_date("2016-04-21", DiaryNode(456, "2016-04-24", "Unspoken Words", "I love you, but I failed to express!", None))
diary.print_diary()

2022-05-05 - Rainy Day
Stayed home and listened to the rain. Peaceful.

2022-05-07 - Weekend Hike
Hiked the Twin Hills trail today. Beautiful views!

2022-05-10 - Coffee Thoughts
Met an old friend over coffee. Unexpected joy.

2024-01-01 - Rainy Night
Met an old friend over coffee. Unexpected joy.

